<a href="https://colab.research.google.com/github/peterij/TM10007_Diagnosing_heart_disease/blob/master/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TM10007 Machine Learning**

In [ ]:
!git clone https://github.com/jveenland/tm10007_ml.git

import zipfile
import os
import pandas as pd

with zipfile.ZipFile('/content/tm10007_ml/ecg/ecg_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/tm10007_ml/ecg')

data = pd.read_csv('/content/tm10007_ml/ecg/ecg_data.csv', index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

Cloning into 'tm10007_ml'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 83 (delta 21), reused 5 (delta 3), pack-reused 31
Unpacking objects: 100% (83/83), 67.93 MiB | 16.67 MiB/s, done.
The number of samples: 827
The number of columns: 9001


In [ ]:
# import function
from sklearn import model_selection


In [ ]:
x_train, x_test = model_selection.train_test_split(data, test_size=0.2)
print(x_train)
print(f'The number of samples: {len(x_train.index)}')
print(f'The number of columns: {len(x_train.columns)}')
# Stratified split met vergelijkbare ratio 

             0_0         0_1         0_2        0_3         0_4         0_5  \
438   771.743658  141.543694   82.458619   8.157553  145.127223   15.586516   
143     2.840660   11.483642    7.498041  13.817329   29.397877   43.719667   
214    47.372007   64.274683    4.086743  10.760164    5.721995   74.508284   
694     6.407810    6.965022   16.757413   9.203876   23.715220  118.225029   
468    10.063743   10.233886   10.508981  15.269568   22.035580   26.597833   
..           ...         ...         ...        ...         ...         ...   
372     0.621266   11.076148   10.845364   8.028119   12.892472   47.594526   
402  5582.270668  273.300453  135.845365  93.299854   68.372794   57.533408   
120     2.916626    3.609774    9.772167   6.267044   13.211785   39.802560   
186     3.554768    0.413673    5.529902  11.764338   19.936010   34.142991   
768     5.796578    4.667601    8.032874   6.104887   18.108585   16.557358   

            0_6        0_7        0_8        0_9  .